<a href="https://colab.research.google.com/github/cerr/pycerr-notebooks/blob/main/autosegment_MR_Rectum_GTV_SMIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Introduction

This tutorial demonstrates deep learning-based auto-segmentation of the GTV on rectal MRIs using the pre-trained SMIT model.

## Requirements

    * Python>=3.8 (for pyCERR)
    * Applying this model requires access to a A100-caliber Tensor Core GPU.

## AI model

    * The segmentation model was trained and validated on Ax T2w Oblique MRI rectal scans.  
    Performance on other MRI sequences and orientations is expected to be sub-optimal.
    
    * The trained model is distributed along with python libraries and other dependencies via a Conda package.

## I/O

* Input: DICOM-formats.
* Output: NIfTI or DICOM RTStruct-format segmentations.
  
Input DICOM images should be organized in directories (one per patient).      

  
  Input dir

          |------Pat1    
                    |------mr_img1.dcm    
                         mr_img2.dcm    
                         ....    
                         ....    
                           
         |-----Pat2    
                 |------mr_img1.dcm    
                        mr_img2.dcm    
                        ....    
                        ....    
                          
## License

By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

```
THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.

This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.
```
You may publish papers and books using results produced using software provided that you reference the appropriate citations:

    SMIT model:  https://arxiv.org/abs/2205.10342
    https://pubmed.ncbi.nlm.nih.gov/39831171/
    CERR library of model implementations: https://doi.org/10.1016/j.ejmp.2020.04.011
    CERR software: https://doi.org/10.1118/1.1568978
    CERR radiomics: https://doi.org/10.1002/mp.13046

YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain such licenses to export, re-export, or import as may be required after delivery to you.


In [ ]:
import glob
import os
import subprocess
import shutil
import numpy as np
import pandas as pd

## I/O paths

In [ ]:
workDir = '/path/to/model_install_dir'
dataDir = '/path/to/mri_dicom_dir'
errLogFile = os.path.join(workDir,'err_log.txt')

sessionPath = r'/paht/to/temp_dir'
if not os.path.exists(sessionPath):
    os.makedirs(sessionPath)

## Download SMIT model

In [ ]:
%%capture
# Downlaod model installer
!git clone https://github.com/cerr/model_installer.git
os.chdir(os.path.join(workDir,'model_installer'))

modelOpt = '10' #MR_Rectum_GTV_SMIT
pythonOpt = 'C' #download and use pre-packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

## Paths to trained model and environment

In [ ]:
# Location of inference script
scriptInstallDir = r'/cluster/home/iyera/model_installer/MR_Rectum_GTV_SMIT'
script1Path = os.path.join(scriptInstallDir,'utils/gen_data_json.py')
script2Path = os.path.join(scriptInstallDir,'run_segmentation_rectum.py')

# Location of Conda archive
condaEnvPath = os.path.join(scriptInstallDir, 'conda-pack')
envActivateScript = os.path.join(condaEnvPath, 'bin', 'activate') # activation script

# Location of model weights
modelPath = os.path.join(scriptInstallDir, 'trained_weights', 'model_final_20251013.pt')

## Map output labels to structure names


In [ ]:
strToLabelMap = {"AI_GTV_SMIT":1}
numLabel = len(strToLabelMap)

## Auto-segment GTVs

In [ ]:
from cerr import plan_container as pc

def importSeg(outputDir, scanNum, planC):
    niiGlob = glob.glob(os.path.join(outputDir, '*.nii.gz'))
    for segFile in niiGlob:
        print('Importing ' + segFile + '...')
        planC = pc.loadNiiStructure(segFile, scanNum, planC, labels_dict=strToLabelMap)
        planC.structure[-1].structureName = "AI_GTV_SMIT"
        print(planC.structure[-1].structureName)

    return planC

In [ ]:
from cerr.dataclasses import scan as cerrScn
from cerr.utils.ai_pipeline import createSessionDir
from cerr.dcm_export import rtstruct_iod

ptList = os.listdir(dataDir)
scanNum = 0

# Loop over MRs
for pt in ptList:
    try:
        # Import MRI to planC
        ptDir = os.path.join(dataDir, pt, timePt)
        print(f'Loading {ptDir}...')
        planC = pc.loadDcmDir(ptDir)
        seriesUID = planC.scan[scanNum].scanInfo[0].seriesInstanceUID
        structFileName = 'AI_seg' + '_' + seriesUID + '.dcm'
        structFilePath = os.path.join(dataPath, structFileName)

        numExistingStructs = len(planC.structure)
        print(f'numExistingStructs: {numExistingStructs}')

        # Create session dir to store temporary data
        modInputPath, modOutputPath = createSessionDir(sessionPath, dataPath)

        # Export scan to NIfTI
        scanFilename = os.path.join(modInputPath,
                                    f"{pt}_scan_3D.nii.gz")
        planC.scan[scanNum].saveNii(scanFilename)

        # Apply model
        print('Beginning inference...')
        cmd = f"source {envActivateScript} && " \
              f"python {script1Path} {modInputPath} && "  \
              f"python {script2Path} --data_dir {modInputPath} "  \
              f"--load_weight_name {modelPath} "  \
              f"--save_folder {modOutputPath} "  \
              f"--out_channels 2 --a_min 0 --a_max 1000 "  \
              f"--space_x 1.5 --space_y 1.5 --space_z 2.0 "  \
              f"--roi_x 128 --roi_y 128 --roi_z 128 --use_smit 1"
        print(cmd)
        subprocess.run(cmd, capture_output=False, shell=True, executable="/bin/bash")
        print('Inference complete.')

        # Import results to planC
        planC = importSeg(modOutputPath, scanNum, planC)
        newNumStructs = len(planC.structure)

        # Remove modOutputPath directory
        shutil.rmtree(modOutputPath)
        os.mkdir(modOutputPath)

        # Export segmentations to DICOM
        print('Exporting to DICOM...')
        seriesDescription = "AI Generated"
        exportOpts = {'seriesDescription': seriesDescription}
        structNumV = np.arange(numExistingStructs, newNumStructs)
        indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID, \
                                                        planC) for structNum in structNumV], dtype=int)
        structsToExportV = structNumV[indOrigV == scanNum]
        rtstruct_iod.create(structsToExportV, structFilePath,
                            planC, exportOpts)

        # Remove temporary session dir
        print('Removing session dir...')
        sessDir, _ = os.path.split(modInputPath)
        shutil.rmtree(sessDir)

    except Exception as e:
        with open(errLogFile, 'a') as file:
            file.write("\n id: {} scan: {} failed with error {}".format(pt, scanNum, e))